In [1]:
import pandas as pd

In [2]:
ies = pd.read_csv('list_ies.txt', header=None)

In [3]:
ies.head(5)

,0,1
0,Alfa - Faculdade de Almenara,NaN
1,Atopp Brasil Faculdade de Negócios - Atopp Bra...,NaN
2,CENTRO UNIVERSITÁRIO SERRA DOS ÓRGÃOS - UNIFESO,NaN
3,Centro Superior de Estudos Jurídicos Carlos Dr...,NaN
4,Centro Técnico-Educacional Superior do Oeste P...,NaN


In [4]:
ies.rename(columns={0: 'ies'}, inplace=True)
ies.drop([1], axis=1, inplace=True)

In [5]:
ies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1232 entries, 0 to 1231
Data columns (total 1 columns):
ies    1232 non-null object
dtypes: object(1)
memory usage: 4.9+ KB


Temos __1.232 IESs__ na lista.

In [6]:
resultado = pd.read_csv('prouni_resultados_000000000000.csv')

In [7]:
resultado.head(5)

,ies,campus,curso,turno,bolsa,candidato,inscricao
0,﻿Centro Universitário São Judas Tadeu - Csjt -...,CAMPUS UNIMONTE,Administração,Matutino,Bolsa Parcial 50%,Bruna Cavarzan Frias de Sant Anna,181040916387
1,Centro Universitário São Judas Tadeu - Csjt - ...,CAMPUS UNIMONTE,Administração,Matutino,Bolsa Parcial 50%,Eduardo Oliveira Santos,181003080437
2,Centro Universitário São Judas Tadeu - Csjt - ...,CAMPUS UNIMONTE,Administração,Matutino,Bolsa Parcial 50%,Gabriele Barbosa Nogueira,181024670620
3,Centro Universitário São Judas Tadeu - Csjt - ...,CAMPUS UNIMONTE,Administração,Matutino,Bolsa Parcial 50%,Giovanna Thompson Lima,181015851221
4,Centro Universitário São Judas Tadeu - Csjt - ...,CAMPUS UNIMONTE,Administração,Matutino,Bolsa Parcial 50%,Julia de Souza Pires Apolinario,181028676565


In [8]:
resultado['ies'].nunique()

2434

Temos __2.434 IESs__ nos resultados. Ou seja, os números não batem. Vamos investigar o motivo.

In [9]:
resultado['ies'].unique()

array(['\ufeffCentro Universitário São Judas Tadeu - Csjt - Csjt',
       'Centro Universitário São Judas Tadeu - Csjt - Csjt',
       '\ufeffAlfa - Faculdade de Almenara', ..., 'Faculdade Seqüencial',
       '\ufeffFaculdade Anhanguera de Valparaíso',
       'Faculdade Anhanguera de Valparaíso'], dtype=object)

Algumas IESs aparecem duplicadas devido à inclusão de __\ufeff__.

In [10]:
resultado['ies'] = resultado['ies'].str.replace('\ufeff', '')

In [11]:
resultado['ies'].nunique()

1227

Após a correção, temos __1.227 IESs__, menos do que a lista. Vejamos quais estão na lista e não nos resultados.

In [12]:
ies_out = ies[~ies['ies'].isin(resultado['ies'])]
ies_out

,ies
166,Christus Faculdade do Piauí - Chrisfapi - Chri...
532,Faculdade Pitágoras -
713,Faculdade Unirb - Ceará
868,Faculdade de Ensino Superior de Marechal Cândi...
1161,Trevisan Escola Superior de Negócios- - Ten - Ten


Aparentemente temos __5 IESs__ que não estão nos resultados. Pode ser que não tenha havido candidado. Checando...

In [13]:
resultado[resultado['ies'].isin(ies_out['ies'])]

,ies,campus,curso,turno,bolsa,candidato,inscricao


Nada. Segunda checagem...

In [14]:
ies_out['ies'].value_counts()

Faculdade Pitágoras -                                                                    1
Christus Faculdade do Piauí - Chrisfapi - Chrisfapi                                      1
Faculdade Unirb - Ceará                                                                  1
Faculdade de Ensino Superior de Marechal Cândido Rondon - Isepe Rondon - Isepe Rondon    1
Trevisan Escola Superior de Negócios- - Ten - Ten                                        1
Name: ies, dtype: int64

In [15]:
resultado[(resultado['ies'] == 'Faculdade de Ensino Superior de Marechal Cândido Rondon - Isepe Rondon - Isepe Rondon') | \
          (resultado['ies'] == 'Faculdade Unirb - Ceará') | \
          (resultado['ies'] == 'Trevisan Escola Superior de Negócios- - Ten - Ten') | \
          (resultado['ies'] == 'Christus Faculdade do Piauí - Chrisfapi - Chrisfapi') | \
          (resultado['ies'] == 'Faculdade Pitágoras -')
         ]

,ies,campus,curso,turno,bolsa,candidato,inscricao


De fato, essas IESs não têm aluno. Portanto, figuram na lista, mas não no resultado.

Validadas as IESs, passamos aos registros gerais.

In [16]:
resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204483 entries, 0 to 204482
Data columns (total 7 columns):
ies          204483 non-null object
campus       204483 non-null object
curso        204483 non-null object
turno        204483 non-null object
bolsa        204483 non-null object
candidato    204483 non-null object
inscricao    204483 non-null int64
dtypes: int64(1), object(6)
memory usage: 6.2+ MB


Temos __204.483 alunos__. Vamos ver se há repetições.

In [17]:
resultado.drop_duplicates(inplace=True)

In [19]:
resultado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204483 entries, 0 to 204482
Data columns (total 7 columns):
ies          204483 non-null object
campus       204483 non-null object
curso        204483 non-null object
turno        204483 non-null object
bolsa        204483 non-null object
candidato    204483 non-null object
inscricao    204483 non-null int64
dtypes: int64(1), object(6)
memory usage: 7.8+ MB


Não há linhas duplicadas.

Vejamos se há `inscricao` duplicada.

In [20]:
resultado[resultado['inscricao'].duplicated(keep='first')]

,ies,campus,curso,turno,bolsa,candidato,inscricao
94772,Centro Universitário Sociesc,PALÁCIO AVENIDA,Administração,Noturno,Bolsa Integral,Aline de Souza Silva,181013017643
94773,Centro Universitário Sociesc,PALÁCIO AVENIDA,Administração,Noturno,Bolsa Integral,Eduarda Westrupp,181007701061
94774,Centro Universitário Sociesc,PALÁCIO AVENIDA,Administração,Noturno,Bolsa Integral,Lia Barbara Maerschner Sausen,181035711546
94775,Centro Universitário Sociesc,PALÁCIO AVENIDA,Administração,Noturno,Bolsa Integral,Pedro Camargo Carrilho Demenjon,181009451764
94776,Centro Universitário Sociesc,PALÁCIO AVENIDA,Administração,Noturno,Bolsa Parcial 50%,Alysson Cardoso Pupo,181007309790
94777,Centro Universitário Sociesc,PALÁCIO AVENIDA,Administração,Noturno,Bolsa Parcial 50%,Beatriz Moro da Silva,181007843137
94778,Centro Universitário Sociesc,PALÁCIO AVENIDA,Administração,Noturno,Bolsa Parcial 50%,Bruno Lodea Rodrigues,181006458556
94779,Centro Universitário Sociesc,PALÁCIO AVENIDA,Administração,Noturno,Bolsa Parcial 50%,Fernanda Elisangela Santos Daldegan,181057044529
94780,Centro Universitário Sociesc,PALÁCIO AVENIDA,Administração,Noturno,Bolsa Parcial 50%,Gabrielle Leticia Conor,181049167446
94781,Centro Universitário Sociesc,PALÁCIO AVENIDA,Administração,Noturno,Bolsa Parcial 50%,Henrique Gabriel Rodrigues,181005746332


Temos __352 alunos__ duplicados. Checando...

In [21]:
resultado[resultado['inscricao'] == 181002501896]

,ies,campus,curso,turno,bolsa,candidato,inscricao
150612,Faculdade de Ciências Sociais Aplicadas - Faci...,UNIDADE SEDE,Radiologia,Noturno,Bolsa Parcial 50%,Maxson dos Santos Gabriel,181002501896
150785,Faculdade de Ciências Sociais Aplicadas -,UNIDADE SEDE,Radiologia,Noturno,Bolsa Parcial 50%,Maxson dos Santos Gabriel,181002501896


In [22]:
resultado[resultado['inscricao'] == 181015111428]

,ies,campus,curso,turno,bolsa,candidato,inscricao
150622,Faculdade de Ciências Sociais Aplicadas - Faci...,UNIDADE SEDE,Serviço Social,Noturno,Bolsa Integral,Patrik Santos Silva,181015111428
150795,Faculdade de Ciências Sociais Aplicadas -,UNIDADE SEDE,Serviço Social,Noturno,Bolsa Integral,Patrik Santos Silva,181015111428


In [23]:
mesma_ies = resultado[resultado['inscricao'] == 181015111428]
mesma_ies['ies'].value_counts()

Faculdade de Ciências Sociais Aplicadas -                    1
Faculdade de Ciências Sociais Aplicadas - Facisa - Facisa    1
Name: ies, dtype: int64

Creio que `Faculdade de Ciências Sociais Aplicadas` e `Faculdade de Ciências Sociais Aplicadas - Facisa - Facisa` sejam a mesma IES. Com isso, não é a inscrição que está duplicada, mas a IES -- apenas apresenta um outro nome.

Vamos ver se temos as duas IESs de maneira distinta na lista.

In [24]:
ies_igual = ies[(ies['ies'] == 'Faculdade de Ciências Sociais Aplicadas - Facisa - Facisa') | \
                (ies['ies'] == 'Faculdade de Ciências Sociais Aplicadas -')]

In [25]:
ies_igual

,ies
807,Faculdade de Ciências Sociais Aplicadas -
808,Faculdade de Ciências Sociais Aplicadas - Faci...


De fato, ambas aparecem na lista; no resultado, portanto, são tratadas como IESs distintas, sendo que é somente uma IES.

Vejamos a consistência de outros campos.

In [26]:
resultado['turno'].value_counts()

Noturno              92598
Curso a Distância    58111
Matutino             41731
Vespertino            7144
Integral              4899
Name: turno, dtype: int64

In [27]:
resultado['bolsa'].value_counts()

Bolsa Integral       113385
Bolsa Parcial 50%     91098
Name: bolsa, dtype: int64

Os demais campos estão consistentes.

Retornando à questão de universidades duplicadas, vamos ver quais são elas.

In [33]:
duplicados = resultado[resultado['inscricao'].duplicated(keep=False)]

In [34]:
duplicados['ies'].value_counts()

Centro Universitário Sociesc                                          177
Centro Universitário Sociesc de Curitiba - Unisociesc - Unisociesc    177
Faculdade de Ciências Sociais Aplicadas -                             173
Faculdade de Ciências Sociais Aplicadas - Facisa - Facisa             173
Centro Universitário Internacional - Uninter - Uninter                  4
Name: ies, dtype: int64

Dois pares têm nomes parecidos e quantidades de duplicados iguais. Uninter é a exceção. Vamos ver por quê.

In [35]:
duplicados[duplicados['ies'] == 'Centro Universitário Internacional - Uninter - Uninter']

,ies,campus,curso,turno,bolsa,candidato,inscricao
140145,Centro Universitário Internacional - Uninter -...,PAP IRANI,Gestão da Produção Industrial,Curso a Distância,Bolsa Integral,Alice Aparecida Alves Tamanho,181012301360
140146,Centro Universitário Internacional - Uninter -...,PAP IRANI,Gestão da Produção Industrial,Curso a Distância,Bolsa Integral,Andre Luis dos Santos Oliveira,181022157133
140147,Centro Universitário Internacional - Uninter -...,PAP IRANI,Gestão da Produção Industrial,Curso a Distância,Bolsa Parcial 50%,Alice Aparecida Alves Tamanho,181012301360
140148,Centro Universitário Internacional - Uninter -...,PAP IRANI,Gestão da Produção Industrial,Curso a Distância,Bolsa Parcial 50%,Andre Luis dos Santos Oliveira,181022157133


Os mesmos alunos foram inscritos duas vezes, cada uma em um tipo de bolsa.